In [19]:
import numpy as np
import argparse
import cv2

In [ ]:
import cv2
import argparse
import numpy as np

# Global variables
frame = None
roiPts = []
inputMode = False

# Mouse callback function to select the ROI points
def selectROI(event, x, y, flags, param):
    global frame, roiPts, inputMode
    # If in ROI selection mode, capture the mouse clicks and draw circles
    if inputMode and event == cv2.EVENT_LBUTTONDOWN and len(roiPts) < 4:
        roiPts.append((x, y))
        cv2.circle(frame, (x, y), 4, (0, 255, 0), 2)
        cv2.imshow("frame", frame)

def main():
    global frame, roiPts, inputMode

    # Start capturing video from the main camera (index 0)
    camera = cv2.VideoCapture(0)

    # Create a window and set the mouse callback to select the ROI
    cv2.namedWindow("frame")
    cv2.setMouseCallback("frame", selectROI)

    # CamShift termination criteria
    termination = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
    roiBox = None
    roiHist = None

    while True:
        (grabbed, frame) = camera.read()
        if not grabbed:
            break

        # If the ROI is selected, track it using CamShift
        if roiBox is not None:
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            backProj = cv2.calcBackProject([hsv], [0], roiHist, [0, 180], 1)
            (r, roiBox) = cv2.CamShift(backProj, roiBox, termination)
            pts = np.int0(cv2.boxPoints(r))  # Update for OpenCV 3+
            cv2.polylines(frame, [pts], True, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("frame", frame)

        # Wait for the user to press a key
        key = cv2.waitKey(1) & 0xFF

        # Press 'i' to select the ROI (enter input mode)
        if key == ord('i') and len(roiPts) < 4:
            inputMode = True
            roiPts = []
            while len(roiPts) < 4:
                cv2.imshow("frame", frame)
                cv2.waitKey(1)
            roiPts = np.array(roiPts)
            s = roiPts.sum(axis=1)
            tl = roiPts[np.argmin(s)]
            br = roiPts[np.argmax(s)]
            roi = frame[tl[1]:br[1], tl[0]:br[0]]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            roiHist = cv2.calcHist([roi], [0], None, [16], [0, 180])
            roiHist = cv2.normalize(roiHist, roiHist, 0, 255, cv2.NORM_MINMAX)
            roiBox = (tl[0], tl[1], br[0] - tl[0], br[1] - tl[1])

        # Exit the loop on pressing 'q'
        elif key == ord('q'):
            break

    # Clean up the camera and close windows
    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


C:\Users\Dharaneesh\AppData\Local\Temp\ipykernel_5588\1266348346.py:44: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(cv2.boxPoints(r))  # Update for OpenCV 3+
